In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from gmplot import GoogleMapPlotter
from random import random

In [15]:
import csv

In [40]:
class CustomGoogleMapPlotter(GoogleMapPlotter):
    def __init__(self, center_lat, center_lng, zoom, apikey='',
                 map_type='satellite'):
        if apikey == '':
            try:
                with open('apikey.txt', 'r') as apifile:
                    apikey = apifile.readline()
            except FileNotFoundError:
                pass
        super().__init__(center_lat, center_lng, zoom, apikey)

        self.map_type = map_type
        assert(self.map_type in ['roadmap', 'satellite', 'hybrid', 'terrain'])

    def write_map(self,  f):
        f.write('\t\tvar centerlatlng = new google.maps.LatLng(%f, %f);\n' %
                (self.center[0], self.center[1]))
        f.write('\t\tvar myOptions = {\n')
        f.write('\t\t\tzoom: %d,\n' % (self.zoom))
        f.write('\t\t\tcenter: centerlatlng,\n')

        # Change this line to allow different map types
        f.write('\t\t\tmapTypeId: \'{}\'\n'.format(self.map_type))

        f.write('\t\t};\n')
        f.write(
            '\t\tvar map = new google.maps.Map(document.getElementById("map_canvas"), myOptions);\n')
        f.write('\n')

    def color_scatter(self, lats, lngs, values=None, colormap='coolwarm',
                      size=None, marker=False, s=None, **kwargs):
        def rgb2hex(rgb):
            """ Convert RGBA or RGB to #RRGGBB """
            rgb = list(rgb[0:3]) # remove alpha if present
            rgb = [int(c * 255) for c in rgb]
            hexcolor = '#%02x%02x%02x' % tuple(rgb)
            return hexcolor

        if values is None:
            colors = [None for _ in lats]
        else:
            cmap = plt.get_cmap(colormap)
            norm = Normalize(vmin=min(values), vmax=max(values))
            scalar_map = ScalarMappable(norm=norm, cmap=cmap)
            colors = [rgb2hex(scalar_map.to_rgba(value)) for value in values]
        for lat, lon, c in zip(lats, lngs, colors):
            self.scatter(lats=[lat], lngs=[lon], c=c, size=size, marker=marker,
                         s=s, **kwargs)

    def cam_tower_scatter(self, lats, lngs, color=None, size=None, marker=True, c=None, s=None, symbol='o',
                          tower_color=None, **kwargs):
        color = color or c
        size = size or s or 40
        kwargs["color"] = color
        kwargs["size"] = size
        settings = self._process_kwargs(kwargs)
        for lat, lng in zip(lats[:-1], lngs[:-1]):
            if marker:
                self.marker(lat, lng, settings['color'])
            else:
                self._add_symbol(Symbol(symbol, lat, lng, size), **settings)
        if marker:
            self.marker(lats[-1], lngs[-1], tower_color)
            tower_col = self.color_dict.get(color)
            tower_col = self.html_color_codes.get(color,color)
            self.points.append((lats[-1], lngs[-1], tower_col[1:], "no implementation"))

In [41]:
#test code to plot all but the last coordinate in list in one color and the last location in a different color

initial_zoom = 12
num_pts = 40

lats = [37.428]
lons = [-122.145]
values = [random() * 20]
for pt in range(num_pts):
    lats.append(lats[-1] + (random() - 0.5)/100)
    lons.append(lons[-1] + random()/100)
    values.append(values[-1] + random())
gmap = CustomGoogleMapPlotter(lats[0], lons[0], initial_zoom,
                              map_type='satellite')
#gmap.color_scatter(lats, lons, values, colormap='gist_heat')
#gmap.scatter(lats,lons,alpha=0.5,c='red',marker=True)
gmap.cam_tower_scatter(lats,lons,alpha=0.5,c='red', marker=True, tower_color='blue')

gmap.draw("mymap_with_tower.html")

In [67]:
cam_lats = []
cam_lons = []
with open('./detection_input_one.csv','r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        cam_lats.append(float(row['lat']))
        cam_lons.append(float(row['lon'])*(-1.))
        #print(row['lat'],row['lon'])
print('cam_lats:',cam_lats,'\ncam_lons:',cam_lons)

cam_lats: [38.596667, 38.597111, 38.597333, 38.597389, 38.597389, 38.597389, 38.597389, 38.597472, 38.597778] 
cam_lons: [-90.640028, -90.640417, -90.640639, -90.640694, -90.640694, -90.640694, -90.640694, -90.640806, -90.641111]


In [33]:
initial_zoom = 12
gmap = CustomGoogleMapPlotter(cam_lats[0], cam_lons[0], initial_zoom,
                              map_type='satellite')
gmap.scatter(cam_lats,cam_lons,alpha=0.5,c='red',marker=True)

gmap.draw("tower_map.html")

In [146]:
tower_lats = []
tower_lons = []
with open('./water_tower_detection.csv','r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        tower_lats.append(float(row['lat']))
        tower_lons.append(float(row['lon'])*(-1.))
print('tower_lats:',tower_lats,'\ntower_lons:',tower_lons)

tower_lats: [38.603696] 
tower_lons: [-90.635958]


In [144]:
all_lats = cam_lats + tower_lats
all_lons = cam_lons + tower_lons

In [145]:
initial_zoom = 12
gmap = CustomGoogleMapPlotter(all_lats[0], all_lons[0], initial_zoom,
                              map_type='satellite')
gmap.cam_tower_scatter(all_lats,all_lons,alpha=0.5,c='red', marker=True, tower_color='blue')
gmap.draw("tower_map_att17.html")

In [59]:
cam_lats = []
cam_lons = []
with open('./detection_input_paper.csv','r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        cam_lats.append(float(row['lat']))
        cam_lons.append(float(row['lon']))
print('cam_lats:',cam_lats,'\ncam_lons:',cam_lons)

cam_lats: [51.509827, 51.509827, 51.509827, 51.509827, 51.509827, 51.509827, 51.509834, 51.50983, 51.50983, 51.50983, 51.50983, 51.509856, 51.509856, 51.509856, 51.509856, 51.509856, 51.509866, 51.509866, 51.509866, 51.509866, 51.50989, 51.50989, 51.50992, 51.50992, 51.509938, 51.510193, 51.510193, 51.510261, 51.510261, 51.510261, 51.510295, 51.510295, 51.510371, 51.510415, 51.510415, 51.510462, 51.510462, 51.510462, 51.510462, 51.510577, 51.510577, 51.510577, 51.510577, 51.510577, 51.510664, 51.510664, 51.510664, 51.511296, 51.511373, 51.511373, 51.511518, 51.511518, 51.511518, 51.511579, 51.511579, 51.511579, 51.511643, 51.511643, 51.511643, 51.511643, 51.511643, 51.511643, 51.511741, 51.511741, 51.511741, 51.511741, 51.511741, 51.5117, 51.5117, 51.5117, 51.5117, 51.511822, 51.511822, 51.511822, 51.511822, 51.511822, 51.511822, 51.511822, 51.511903, 51.511903, 51.511903, 51.511903, 51.511903, 51.511903, 51.511946, 51.511946, 51.511987, 51.511987, 51.511987, 51.511987, 51.511987, 51.5

In [60]:
initial_zoom = 12
gmap = CustomGoogleMapPlotter(cam_lats[0], cam_lons[0], initial_zoom,
                              map_type='satellite')
gmap.scatter(cam_lats,cam_lons,alpha=0.5,c='red',marker=True)

gmap.draw("paper_map.html")